In [20]:
import csv
from datetime import datetime
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Define the stations with their labels and coordinates
stations = [
    {"label": "VSKP", "latitude": 17.6883, "longitude": 83.2185},
    {"label": "DVD", "latitude": 17.0150, "longitude": 82.2321},
    {"label": "AKP", "latitude": 16.9849, "longitude": 81.7866},
    # Add all other stations with their latitudes and longitudes...
]

# Example raw data with train delays for each date
data = [
    [datetime(2023, 11, 1), 1, 0, 2],  # Example delays for VSKP, DVD, AKP
    [datetime(2023, 11, 2), 0, 1, 3],
    # Add more rows as needed
]

# Prepare the initial table with just dates and delays
dates = [row[0].strftime('%d/%m/%y') for row in data]
df = pd.DataFrame({"Date": dates})

# Add delay data for each station in the format specified
for i, station in enumerate(stations):
    df[station["label"]] = [row[i + 1] for row in data]

# Define columns in the required order and retrieve weather data
for station in stations:
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": station["latitude"],
        "longitude": station["longitude"],
        "start_date": "2023-11-01",
        "end_date": "2023-11-02",  # Adjusted end date based on data range
        "daily": ["weather_code", "temperature_2m_mean", "wind_speed_10m_max"]
    }

    # Fetch data from API
    response = openmeteo.weather_api(url, params=params)[0]
    daily = response.Daily()
    dates = pd.date_range(
        start=pd.to_datetime(daily.Time(), unit="s", utc=True),
        end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=daily.Interval()),
        inclusive="left"
    )

    # Ensure dates is a list, matching the length of weather data variables
    dates = list(dates)
    weather_code = daily.Variables(0).ValuesAsNumpy()
    temperature = daily.Variables(1).ValuesAsNumpy()
    wind_speed = daily.Variables(2).ValuesAsNumpy()

    # Create a temporary DataFrame to store the data for this station
    temp_df = pd.DataFrame({
        "Date": [date.strftime('%d/%m/%y') for date in dates],
        f"{station['label']}_Delay": df[station["label"]].values,
        f"weather_code_{station['label']}": weather_code,
        f"Temperature_{station['label']}": temperature,
        f"Wind_Speed_{station['label']}": wind_speed,
        f"Distance_travelled_{station['label']}": [None] * len(weather_code)  # Placeholder for Distance
    })

    # Merge this station's data back into the main DataFrame
    df = df.merge(temp_df, on="Date", how="left")

# Drop the original delay columns used for initialization
df.drop(columns=[station["label"] for station in stations], inplace=True)

# Save final table with delays and weather details
df.to_csv("merged_train_weather_data.csv", index=False)
print("Data has been formatted and saved to 'merged.csv'")


Data has been formatted and saved to 'merged.csv'
